### Data Collection:

Publicly Available Data Sources:
S&P Case-Shiller Home Price Index (CSUSHPISA): Provides historical home price data.
Federal Reserve Economic Data (FRED): A comprehensive source for economic indicators, including GDP, unemployment rates, and more.
Bureau of Economic Analysis (BEA): For data on GDP and consumer price index (CPI).
Census Bureau: For housing and population statistics.
Key Factors:
Consumer Price Index (CPI): Measures changes in the price level of a basket of consumer goods and services, influencing the cost of living and home prices.
Per Capita GDP: Indicates economic prosperity which can impact home prices.
Unemployment Rate (UNRATE): Reflects the health of the job market, influencing housing demand.
Federal Funds Rate (FEDFUNDS): Affects mortgage rates and thus home prices.
Housing Supply (Houses): The number of houses available for sale affects housing prices.
Income Levels: Higher median income generally increases home buying power.
Data Preprocessing:

Cleaning: Remove or handle missing values, incorrect data, and outliers.
Feature Engineering: Create relevant features from the raw data.
Model Building:

Feature Selection: Identify and select key features that influence home prices.
Training: Train various machine learning models (e.g., Linear Regression, Random Forest, XGBoost) to predict home prices based on the selected features.
Evaluation: Assess model performance using metrics such as Mean Squared Error (MSE) and R-squared.
Analysis of Feature Importance:

Use models that provide feature importance (e.g., Random Forest, XGBoost) to determine which factors most influence home prices.

Data for all the variables is downloaded, preprocessed, and combined to create a dataset using the **Extract Transform Load (ETL)** method. Data for different variables had different frequencies. So, to combine the data, the necessary interpolations are made.



#### Importing neccessary libraries

In [ ]:
import numpy as np
import pandas as pd

#### Perform ETL

In [ ]:
# Reading CASE-SHILLER Index into a dataframe
df_CS = pd.read_csv("CSUSHPISA.csv")

# Changing dtype of date column
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])

# Selecting data till JULY 2023
mask = df_CS["DATE"] <= "2023-07-01"
df_CS = df_CS[mask]

#Resetting Index
df_CS.reset_index(inplace = True)
df_CS.drop(columns = ["index"], inplace = True)

# Creating "Year" and "Month" columns
df_CS["Year"] = pd.DatetimeIndex(df_CS["DATE"]).year
df_CS["Month"] = pd.DatetimeIndex(df_CS["DATE"]).month
print("Shape of the CASE-SHILLER Index:- ", df_CS.shape)
df_CS.tail()


Shape of the CASE-SHILLER Index:-  (439, 4)


,DATE,CSUSHPINSA,Year,Month
434,2023-03-01,297.316,2023,3
435,2023-04-01,301.469,2023,4
436,2023-05-01,305.427,2023,5
437,2023-06-01,308.307,2023,6
438,2023-07-01,310.162,2023,7


In [ ]:
# Reading Unemployment Rate Data into a dataframe
df_unemp = pd.read_csv("UNRATE.csv")
df_unemp.drop([259], inplace = True)
print("Unemployment Rate Data:- ", df_unemp.shape)
df_unemp.tail()

Unemployment Rate Data:-  (259, 2)


,DATE,UNRATE
254,2023-03-01,3.5
255,2023-04-01,3.4
256,2023-05-01,3.7
257,2023-06-01,3.6
258,2023-07-01,3.5


In [ ]:
# Reading Employment Rate Data into a dataframe
df_emp = pd.read_csv("Employment_rate.csv")
df_emp = df_emp.rename(columns={'LREM64TTUSM156S': 'EmpRate'})
df_emp.drop([259], inplace = True)
print("shape of the Employment Rate Data:- ", df_emp.shape)
df_emp.tail()

shape of the Employment Rate Data:-  (569, 2)


,DATE,EmpRate
565,2024-02-01,71.88552
566,2024-03-01,72.00176
567,2024-04-01,72.02491
568,2024-05-01,71.88110
569,2024-06-01,71.82606


In [ ]:
# Reading Per Capita GDP Data into a dataframe
df_pcgdp = pd.read_csv("Per_Capita_GDP.csv", names = ["DATE", "A939RX0Q048SBEA"], skiprows = 1)
df_pcgdp = df_pcgdp.rename(columns={'A939RX0Q048SBEA': 'Per_Capita_GDP'})
print("Shape of the Per Capita GDP Data:- ", df_pcgdp.shape)
df_pcgdp.tail()

Shape of the Per Capita GDP Data:-  (309, 2)


,DATE,Per_Capita_GDP
304,2023-01-01,66096.0
305,2023-04-01,66357.0
306,2023-07-01,67050.0
307,2023-10-01,67513.0
308,2024-01-01,67672.0


The data is quarterly. We will impute for other months using linear interpolation after we create the final dataframe combining all the data.


In [ ]:
# Interest Rate Data
df_Fed_rate = pd.read_csv("Interest_rates.csv").drop([259])
print("Shape of the Interest rate data:- ",df_Fed_rate.shape)
df_Fed_rate.tail()

Shape of the Interest rate data:-  (259, 2)


,DATE,FEDFUNDS
254,2023-03-01,4.65
255,2023-04-01,4.83
256,2023-05-01,5.06
257,2023-06-01,5.08
258,2023-07-01,5.12


In [ ]:
# Reading Construction Material Data into a dataframe
df_cons_price_index = pd.read_csv("Construction_price_index.csv", names = ["DATE", "WPUSI012011"], skiprows = 1)
df_cons_price_index = df_cons_price_index.rename(columns={'WPUSI012011': 'Cons_Material'})
df_cons_price_index.drop([259], inplace = True)
print("Shape of the Construction Material Data:- ", df_cons_price_index.shape)
df_cons_price_index.tail()

Shape of the Construction Material Data:-  (259, 2)


,DATE,Cons_Material
254,2023-03-01,331.729
255,2023-04-01,333.366
256,2023-05-01,337.473
257,2023-06-01,337.278
258,2023-07-01,334.449


In [ ]:
# Consumer Price Index
df_CPI = pd.read_csv("Consumer_price_index.csv", names = ["DATE", "CPIAUCSL"], skiprows = 1).drop([259])
df_CPI = df_CPI.rename(columns={'CPIAUCSL': 'CPI'})
print("Shape of the Consumer Price Index:- ", df_CPI.shape)
df_CPI.tail()

Shape of the Consumer Price Index:-  (259, 2)


,DATE,CPI
254,2023-03-01,301.808
255,2023-04-01,302.918
256,2023-05-01,303.294
257,2023-06-01,303.841
258,2023-07-01,304.348


In [ ]:
# Monthly new house supply
df_house = pd.read_csv("monthly_house_supply.csv", names = ["DATE", "MSACSR"], skiprows = 1).drop([259])
df_house = df_house.rename(columns={'MSACSR': 'Houses'})
print("Shape of the monthly house supply data:- ", df_house.shape)
df_house.tail()


Shape of the monthly house supply data:-  (259, 2)


,DATE,Houses
254,2023-03-01,8.1
255,2023-04-01,7.6
256,2023-05-01,7.2
257,2023-06-01,7.5
258,2023-07-01,7.0


In [ ]:
# Population above 65

df_oldpop = pd.read_csv("old_age_pop.csv", names = ["DATE", "old_age_pop"], skiprows = 1)
df_oldpop['DATE'] = pd.to_datetime(df_oldpop['DATE'], format="%d-%m-%Y").dt.strftime("%Y-%m-%d")
print("Shape of the population data age above 65:- ", df_oldpop.shape)
df_oldpop.tail()

Shape of the population data age above 65:-  (21, 2)


,DATE,old_age_pop
16,2018-01-01,15.4
17,2019-01-01,15.8
18,2020-01-01,16.2
19,2021-01-01,16.7
20,2022-01-01,17.1


In [ ]:
# Urban Population Percent

df_urban = pd.read_csv("urban_pop.csv")
df_urban['DATE'] = pd.to_datetime(df_urban['DATE'], format="%d-%m-%Y").dt.strftime("%Y-%m-%d")
print("Shape of the urban population percent data:- ", df_urban.shape)
df_urban.tail()

Shape of the urban population percent data:-  (21, 2)


,DATE,urban_pop_us
16,2018-01-01,82.3
17,2019-01-01,82.5
18,2020-01-01,82.7
19,2021-01-01,82.9
20,2022-01-01,83.1


In [ ]:
# Housing Subsidies

df_subsidy = pd.read_csv("housing_subsidies.csv", names = ["DATE", "Subsidy"], skiprows = 1)
print("Shape of the housing subsidies:- ", df_subsidy.shape)
df_subsidy.tail()


Shape of the housing subsidies:-  (21, 2)


,DATE,Subsidy
16,2018-01-01,38.859
17,2019-01-01,40.185
18,2020-01-01,44.147
19,2021-01-01,45.299
20,2022-01-01,48.021


In [ ]:
# Working age population

df_working = pd.read_csv("working_age_population.csv", names = ["DATE", "LFWA64TTUSM647S"], skiprows = 1).drop([259])
df_working = df_working.rename(columns={'LFWA64TTUSM647S': 'working_age_pop'})
print("Shape of the working age population:- ", df_working.shape)
df_working.tail()

Shape of the working age population:-  (259, 2)


,DATE,working_age_pop
254,2023-03-01,2.082729e+08
255,2023-04-01,2.083833e+08
256,2023-05-01,2.086119e+08
257,2023-06-01,2.087165e+08
258,2023-07-01,2.087998e+08


In [ ]:
# Real Median Household Income

df_income = pd.read_csv("median_household_income.csv", names = ["DATE", "MEHOINUSA672N"], skiprows = 1)
df_income = df_income.rename(columns={'MEHOINUSA672N': 'median_income'})
print("Shape of the median household income data:- ", df_income.shape)
df_income.tail()


Shape of the median household income data:-  (21, 2)


,DATE,median_income
16,2018-01-01,73030
17,2019-01-01,78250
18,2020-01-01,76660
19,2021-01-01,76330
20,2022-01-01,74580


In [ ]:
# Total number of households

df_households = pd.read_csv("household.csv", names = ["DATE", "TTLHH"], skiprows = 1)
df_households = df_households.rename(columns={'TTLHH': 'Num_Households'})
print("Shape of the total households data:- ", df_households.shape)
df_households.tail()


Shape of the total households data:-  (21, 2)


,DATE,Num_Households
16,2018-01-01,127586.0
17,2019-01-01,128579.0
18,2020-01-01,128451.0
19,2021-01-01,129224.0
20,2022-01-01,131202.0


In [ ]:
# Merging Per Capita GDP (Quarterly data)
df_pcgdp["DATE"] = pd.to_datetime(df_pcgdp["DATE"])
df_CS = pd.merge(df_CS,df_pcgdp, how = "left")
df_CS.head()


,DATE,CSUSHPISA,Year,Month,Per_Capita_GDP
0,2002-01-01,117.144,2002,1,50091.0
1,2002-02-01,117.845,2002,2,NaN
2,2002-03-01,118.687,2002,3,NaN
3,2002-04-01,119.611,2002,4,50286.0
4,2002-05-01,120.724,2002,5,NaN


In [ ]:
# Concating dataframes having monthly data to create one dataframe
df = pd.DataFrame()
df_bymonth = [df_CS, df_working, df_house, df_CPI, df_unemp, df_emp, df_cons_price_index, df_Fed_rate]
for df1 in df_bymonth:
    df1["DATE"] = pd.to_datetime(df1["DATE"])
    df1 = df1.set_index("DATE")
    df = pd.concat([df,df1], axis = 1)
print(df.shape)
df.head()


(259, 11)


,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS
DATE,,,,,,,,,,,
2002-01-01,117.144,2002,1,50091.0,1.825653e+08,4.2,177.7,5.7,72.016492,142.0,1.73
2002-02-01,117.845,2002,2,NaN,1.827970e+08,4.0,178.0,5.7,72.333310,142.2,1.74
2002-03-01,118.687,2002,3,NaN,1.830774e+08,4.1,178.5,5.7,72.132190,143.2,1.73
2002-04-01,119.611,2002,4,50286.0,1.832597e+08,4.3,179.3,5.9,71.915086,143.5,1.75
2002-05-01,120.724,2002,5,NaN,1.835085e+08,4.0,179.5,5.8,72.025945,143.8,1.75


In [ ]:
# Merging other dataframes
others = [df_urban, df_households, df_income, df_subsidy, df_oldpop]
for df1 in others:
    if "Year" not in df1.columns:
        df1["Year"] = pd.DatetimeIndex(df1["DATE"]).year
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
    else:
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
df["DATE"] = df_CS["DATE"]
df.set_index("DATE", inplace = True)
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,urban_pop_us,Num_Households,median_income,Subsidy,old_age_pop
DATE,,,,,,,,,,,,,,,,
2002-01-01,117.144,2002,1,50091.0,1.825653e+08,4.2,177.7,5.7,72.016492,142.0,1.73,79.4,109297.0,65820.0,24.183,12.3
2002-02-01,117.845,2002,2,NaN,1.827970e+08,4.0,178.0,5.7,72.333310,142.2,1.74,79.4,109297.0,65820.0,24.183,12.3
2002-03-01,118.687,2002,3,NaN,1.830774e+08,4.1,178.5,5.7,72.132190,143.2,1.73,79.4,109297.0,65820.0,24.183,12.3
2002-04-01,119.611,2002,4,50286.0,1.832597e+08,4.3,179.3,5.9,71.915086,143.5,1.75,79.4,109297.0,65820.0,24.183,12.3
2002-05-01,120.724,2002,5,NaN,1.835085e+08,4.0,179.5,5.8,72.025945,143.8,1.75,79.4,109297.0,65820.0,24.183,12.3


In [ ]:
print(df.shape)

(259, 16)


Check missing values (NAN)

In [ ]:
df.isna().sum()

CSUSHPISA            0
Year                 0
Month                0
Per_Capita_GDP     172
working_age_pop      0
Houses               0
CPI                  0
UNRATE               0
EmpRate              0
Cons_Material        0
FEDFUNDS             0
urban_pop_us         7
Num_Households       7
median_income        7
Subsidy              7
old_age_pop          7
dtype: int64

You can see "Per_Capita_GDP" column has missing values because the data was quarterly. The missing values in the other columns are due to the unavailability of fresh data. We will first fill in the missing values in the "Per_Capita_GDP" column using linear interpolation. We will drop the rows with missing values in the other columns. This means that we will use data from 2002 to 2022.
Interpolation is a mathematical technique used to estimate values that are missing in a dataset based on the values of neighboring data points. It calculates intermediate values based on the existing data.



In [ ]:
# Filling missing values in the Per_Capita_GDP column using linear interpolation
df["Per_Capita_GDP"] = df["Per_Capita_GDP"].interpolate()

In [ ]:
df

,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,urban_pop_us,Num_Households,median_income,Subsidy,old_age_pop
DATE,,,,,,,,,,,,,,,,
2002-01-01,117.144,2002,1,50091.000000,1.825653e+08,4.2,177.700,5.7,72.016492,142.000,1.73,79.4,109297.0,65820.0,24.183,12.3
2002-02-01,117.845,2002,2,50156.000000,1.827970e+08,4.0,178.000,5.7,72.333310,142.200,1.74,79.4,109297.0,65820.0,24.183,12.3
2002-03-01,118.687,2002,3,50221.000000,1.830774e+08,4.1,178.500,5.7,72.132190,143.200,1.73,79.4,109297.0,65820.0,24.183,12.3
2002-04-01,119.611,2002,4,50286.000000,1.832597e+08,4.3,179.300,5.9,71.915086,143.500,1.75,79.4,109297.0,65820.0,24.183,12.3
2002-05-01,120.724,2002,5,50311.333333,1.835085e+08,4.0,179.500,5.8,72.025945,143.800,1.75,79.4,109297.0,65820.0,24.183,12.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-01,298.539,2023,3,66253.333333,2.082729e+08,8.1,301.808,3.5,71.893933,331.729,4.65,NaN,NaN,NaN,NaN,NaN
2023-04-01,300.198,2023,4,66341.000000,2.083833e+08,7.6,302.918,3.4,71.982434,333.366,4.83,NaN,NaN,NaN,NaN,NaN
2023-05-01,302.620,2023,5,66573.666667,2.086119e+08,7.2,303.294,3.7,71.937953,337.473,5.06,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.dropna(inplace = True)

In [ ]:
df.isna().sum()

CSUSHPISA          0
Year               0
Month              0
Per_Capita_GDP     0
working_age_pop    0
Houses             0
CPI                0
UNRATE             0
EmpRate            0
Cons_Material      0
FEDFUNDS           0
urban_pop_us       0
Num_Households     0
median_income      0
Subsidy            0
old_age_pop        0
dtype: int64

In [ ]:
df

,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,urban_pop_us,Num_Households,median_income,Subsidy,old_age_pop
DATE,,,,,,,,,,,,,,,,
2002-01-01,117.144,2002,1,50091.000000,1.825653e+08,4.2,177.700,5.7,72.016492,142.000,1.73,79.4,109297.0,65820.0,24.183,12.3
2002-02-01,117.845,2002,2,50156.000000,1.827970e+08,4.0,178.000,5.7,72.333310,142.200,1.74,79.4,109297.0,65820.0,24.183,12.3
2002-03-01,118.687,2002,3,50221.000000,1.830774e+08,4.1,178.500,5.7,72.132190,143.200,1.73,79.4,109297.0,65820.0,24.183,12.3
2002-04-01,119.611,2002,4,50286.000000,1.832597e+08,4.3,179.300,5.9,71.915086,143.500,1.75,79.4,109297.0,65820.0,24.183,12.3
2002-05-01,120.724,2002,5,50311.333333,1.835085e+08,4.0,179.500,5.8,72.025945,143.800,1.75,79.4,109297.0,65820.0,24.183,12.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-01,301.596,2022,8,65569.000000,2.073941e+08,8.7,295.320,3.7,71.466900,342.753,2.33,83.1,131202.0,74580.0,48.021,17.1
2022-09-01,299.380,2022,9,65676.000000,2.074743e+08,9.7,296.539,3.5,71.444231,336.464,2.56,83.1,131202.0,74580.0,48.021,17.1
2022-10-01,298.922,2022,10,65783.000000,2.074563e+08,9.7,297.987,3.7,71.247799,333.796,3.08,83.1,131202.0,74580.0,48.021,17.1


In [ ]:
print("Shape of the dataframe after preprocessing:- ", df.shape)

Shape of the dataframe after preprocessing:-  (252, 16)


This is our preprocessed datset. Let's save it as "prepared_dataset.csv".


In [ ]:
df.to_csv("prepared_dataset.csv")

In [ ]:
us_house_price_df = pd.read_csv("prepared_dataset.csv").set_index("DATE")
us_house_price_df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,urban_pop_us,Num_Households,median_income,Subsidy,old_age_pop
DATE,,,,,,,,,,,,,,,,
2002-01-01,117.144,2002,1,50091.000000,1.825653e+08,4.2,177.7,5.7,72.016492,142.0,1.73,79.4,109297.0,65820.0,24.183,12.3
2002-02-01,117.845,2002,2,50156.000000,1.827970e+08,4.0,178.0,5.7,72.333310,142.2,1.74,79.4,109297.0,65820.0,24.183,12.3
2002-03-01,118.687,2002,3,50221.000000,1.830774e+08,4.1,178.5,5.7,72.132190,143.2,1.73,79.4,109297.0,65820.0,24.183,12.3
2002-04-01,119.611,2002,4,50286.000000,1.832597e+08,4.3,179.3,5.9,71.915086,143.5,1.75,79.4,109297.0,65820.0,24.183,12.3
2002-05-01,120.724,2002,5,50311.333333,1.835085e+08,4.0,179.5,5.8,72.025945,143.8,1.75,79.4,109297.0,65820.0,24.183,12.3


**This data which is processed is taken to Model training......**